### 0. Import Lib

In [40]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from ipywidgets import interact

### 1. Draw bbox with Interact

In [41]:
image_path = "/data/HiNAS-DATA/MassMIND/Images/"
semantic_mask_path = "/data/HiNAS-DATA/MassMIND/Segmentation_Masks/"
instance_mask_path = "/data/HiNAS-DATA/MassMIND/InstanceSegmentation_Masks/"

# Get the list of all files in the instance_mask_path directory
files = os.listdir(instance_mask_path)

@interact(idx=(0,len(files)-1))
def draw_mask(idx=0):
    # Get the file name without extension
    file_name = os.path.splitext(files[idx])[0]
    
    # Load the image, semantic mask, and instance mask
    image = cv2.imread(image_path + file_name + '.png', cv2.IMREAD_UNCHANGED)
    semantic_mask = cv2.imread(semantic_mask_path + file_name + '.png', cv2.IMREAD_UNCHANGED)
    instance_mask = cv2.imread(instance_mask_path + file_name + '.png', cv2.IMREAD_UNCHANGED)
    
    # print(f"semantic mask min: {np.min(semantic_mask)}, max: {np.max(semantic_mask)}")
    # print(f"instance mask min: {np.min(instance_mask)}, max: {np.max(instance_mask)}")
    # print(f"semantic_mask size: {semantic_mask.shape}, instance_mask size: {instance_mask.shape} ")
    values = np.unique(instance_mask).tolist()
    
    # Loop over the values between min_value and max_value inclusive
    for i in values:
        # Create a binary mask for the current value
        mask = (instance_mask == i)
        
        # Extract the corresponding pixels in the semantic mask
        semantic_pixels_mask = semantic_mask[mask]
        
        # Count the number of pixels with a value of 3
        semantic_pixels = np.sum(semantic_pixels_mask == 3)
        
        mask_pixels = np.sum(mask)
        
        # If 80% of the pixels have a value of 2 in the semantic mask, extract the bounding box
        if semantic_pixels/mask_pixels == 1.0:
        
            # Convert the mask to a binary image
            mask_binary = (mask > 0).astype(np.uint8) * 255

            # Find the contours in the binary image
            contours, _ = cv2.findContours(mask_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            x, y, w, h = cv2.boundingRect(contours[0])
            
            # Draw the bounding box on the image
            cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    # Display the image with the bounding box
    plt.figure(figsize=(16,12))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()


interactive(children=(IntSlider(value=0, description='idx', max=2915), Output()), _dom_classes=('widget-intera…

### 2. Save yolo bbox file

In [42]:

# Set the paths to the data directories
image_path = "/data/HiNAS-DATA/MassMIND/Images/"
semantic_mask_path = "/data/HiNAS-DATA/MassMIND/Segmentation_Masks/"
instance_mask_path = "/data/HiNAS-DATA/MassMIND/InstanceSegmentation_Masks/"
yolo_label_path = "/data/HiNAS-DATA/MassMIND/labels/"

# Create the yolo_label_path directory if it doesn't exist
if not os.path.exists(yolo_label_path):
    os.makedirs(yolo_label_path)

# Get the list of all files in the instance_mask_path directory
files = os.listdir(instance_mask_path)

# Loop over the files in the instance_mask_path directory
for file_name in files:
    # Get the file name without extension
    file_name_without_ext = os.path.splitext(file_name)[0]

    # Load the image, semantic mask, and instance mask
    image = cv2.imread(image_path + file_name_without_ext + '.png', cv2.IMREAD_UNCHANGED)
    semantic_mask = cv2.imread(semantic_mask_path + file_name_without_ext + '.png', cv2.IMREAD_UNCHANGED)
    instance_mask = cv2.imread(instance_mask_path + file_name_without_ext + '.png', cv2.IMREAD_UNCHANGED)

    # Get the unique values in the instance mask
    values = np.unique(instance_mask).tolist()

    # Create a list to store the bbox labels
    bbox_labels = []

    # Loop over the values in the instance mask
    for i in values:
        # Create a binary mask for the current value
        mask = (instance_mask == i)

        # Extract the corresponding pixels in the semantic mask
        semantic_pixels_mask = semantic_mask[mask]

        # Count the number of pixels with a value of 3
        semantic_pixels = np.sum(semantic_pixels_mask == 3)

        mask_pixels = np.sum(mask)

        # If 80% of the pixels have a value of 2 in the semantic mask, extract the bounding box
        if semantic_pixels/mask_pixels == 1.0:
            # Find the contours in the mask
            contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            # Get the bbox coordinates
            x, y, w, h = cv2.boundingRect(contours[0])

            # Calculate the center coordinates and dimensions of the bbox
            center_x = (x + w/2) / image.shape[1]
            center_y = (y + h/2) / image.shape[0]
            bbox_width = w / image.shape[1]
            bbox_height = h / image.shape[0]

            # Add the bbox label to the list
            bbox_label = f"{i} {center_x:.6f} {center_y:.6f} {bbox_width:.6f} {bbox_height:.6f}"
            bbox_labels.append(bbox_label)

    # Write the bbox labels to a file in the yolo_label_path directory
    with open(os.path.join(yolo_label_path, file_name_without_ext + '.txt'), 'w') as f:
        f.write('\n'.join(bbox_labels))
